In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name, version=None):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check along with specific versions if necessary
packages = [
    {"name": "tqdm", "version": None},
    {"name": "gdown", "version": None},
    {"name": "numpy", "version": "1.23.5"},
    {"name": "pandas", "version": None}  # Added pandas
]

# Check and install packages
for package in packages:
    check_and_install_package(package["name"], package["version"])



tqdm is already installed.

gdown is already installed.

numpy is already installed.

pandas is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Feature_Engineered_DF.parquet' /content/

# Load the parquet file into a pandas DataFrame
output_path = '/content/Feature_Engineered_DF.parquet'
df = pd.read_parquet(output_path)

print("The Feature Engineered DataFrame has been loaded successfully.")


The Feature Engineered DataFrame has been loaded successfully.


In [ ]:
df.head()

,fuel_type,body_type,city,city_fuel_economy,days_in_market,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,...,major_options_count,hp_x_engine_disp,hp_x_torque,listed_day,listed_month,listed_year,age,resale_value_score,maintenance_cost,luxury_score
0,Gasoline,SUV / Crossover,Ontario,20.000000,23,91761,3500.0,V6,Blue,True,...,5,0.18,-0.02369,20,8,2020,0,32,41,33
1,Gasoline,Sedan,Elizabeth,22.000000,22,07202,2000.0,I4,Black,False,...,8,-0.04,0.00000,18,8,2020,3,26,38,36
2,Biodiesel,Pickup Truck,Omaha,22.690001,93,68134,6700.0,V8,White,True,...,5,7.40,0.00003,9,6,2020,0,30,46,34
3,Gasoline,SUV / Crossover,Clearwater,22.690001,163,33763,1500.0,I4,Black,True,...,5,0.97,0.50512,31,3,2020,0,30,37,32
4,Gasoline,SUV / Crossover,Chillicothe,18.000000,25,64601,3000.0,V6,Blue,True,...,15,0.08,0.00002,16,8,2020,0,28,45,39




---



## **Linear Regression**

In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ignore warnings
warnings.filterwarnings('ignore')

# Start tracking overall runtime
start_time = time.time()

# Combine processing data and model training in the same progress bar
with tqdm(total=6, desc="Processing and Training") as pbar:

    # Sample 20% of the data
    df_sample = df.sample(frac=0.4, random_state=42)
    pbar.update(1)

    # Handle categorical columns
    cat_columns = df_sample.select_dtypes(include=['object']).columns.tolist()
    num_columns = df_sample.select_dtypes(exclude=['object']).columns.tolist()
    num_columns.remove('price')  # Exclude the target column 'price'
    pbar.update(1)

    # Convert 'franchise_dealer' to numeric if necessary
    df_sample['franchise_dealer'] = df_sample['franchise_dealer'].astype(int)

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_columns)
        ]
    )
    pbar.update(1)

    # Split the data into training and test sets
    X = df_sample.drop(columns='price')
    y = df_sample['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pbar.update(1)

    # Apply the preprocessing pipeline
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    pbar.update(1)

    # Train Linear Regression model
    lr = LinearRegression()
    model = lr.fit(X_train_transformed, y_train)
    pbar.update(1)

# Make predictions
print("Making predictions...")
y_pred = model.predict(X_test_transformed)

# Evaluate the model
r2 = r2_score(y_test, y_pred)

# Display results
print(f"\nTrain size: {len(X_train):,} samples")
print(f"Test size: {len(X_test):,} samples")

# Multiply R-Squared by 100 for percentage calculation
print(f"\n\nR-Squared Score (Accuracy): {r2 * 100:.2f}%")

# Calculate total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60  # Convert seconds to minutes

print(f"\nOverall runtime: {round(total_runtime)} minutes.")

# Calculate additional metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Output additional metrics
print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")


Processing and Training: 100%|██████████| 6/6 [13:25<00:00, 134.32s/it]

Making predictions...

Train size: 960,012 samples
Test size: 240,004 samples


R-Squared Score (Accuracy): 81.86%

Overall runtime: 13 minutes.
Additional Metrics:
Mean Absolute Error: 4059
Mean Squared Error: 65651380
Root Mean Squared Error: 8103


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from tqdm import tqdm
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ignore warnings
warnings.filterwarnings('ignore')

# Start tracking overall runtime
start_time = time.time()

# Combine processing data and model training in the same progress bar
with tqdm(total=6, desc="Processing and Training") as pbar:

    # Sample 40% of the data
    df_sample = df.sample(frac=0.4, random_state=42)
    pbar.update(1)

    # Handle categorical columns
    cat_columns = df_sample.select_dtypes(include=['object']).columns.tolist()
    num_columns = df_sample.select_dtypes(exclude=['object']).columns.tolist()
    num_columns.remove('price')  # Exclude the target column 'price'
    pbar.update(1)

    # Convert 'franchise_dealer' to numeric if necessary
    df_sample['franchise_dealer'] = df_sample['franchise_dealer'].astype(int)

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_columns)
        ]
    )
    pbar.update(1)

    # Split the data into training and test sets
    X = df_sample.drop(columns='price')
    y = df_sample['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pbar.update(1)

    # Apply the preprocessing pipeline
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    pbar.update(1)

# Define the parameter grid for LinearRegression
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}

# Custom scorer that prints results immediately along with parameters
def custom_scorer(y_true, y_pred, params):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Print the current parameters and their corresponding metrics
    print(f"\nParameters: {params}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f"R-Squared Score (Accuracy): {r2 * 100:.2f}%")

    return r2  # Return R-squared as the score

# Perform manual grid search
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid):
    lr = LinearRegression(**params)

    # Train the model
    model = lr.fit(X_train_transformed.toarray(), y_train)

    # Make predictions
    y_pred = model.predict(X_test_transformed.toarray())

    # Score and print parameters and metrics
    score = custom_scorer(y_test, y_pred, params)

    # Update best score and parameters if the current score is better
    if score > best_score:
        best_score = score
        best_params = params

# Print best parameters
print(f"\nBest Parameters: {best_params}")
print(f"Best R-Squared Score: {best_score * 100:.2f}%")

# Train final model with best parameters
best_model = LinearRegression(**best_params)
best_model.fit(X_train_transformed, y_train)

# Final evaluation
y_pred = best_model.predict(X_test_transformed)
final_r2 = r2_score(y_test, y_pred)
final_mae = mean_absolute_error(y_test, y_pred)
final_mse = mean_squared_error(y_test, y_pred)
final_rmse = np.sqrt(final_mse)

print("\nFinal Model Metrics:")
print(f"R-Squared Score (Accuracy): {final_r2 * 100:.2f}%")
print(f"Mean Absolute Error: {final_mae:.2f}")
print(f"Mean Squared Error: {final_mse:.2f}")
print(f"Root Mean Squared Error: {final_rmse:.2f}")

# Calculate total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60  # Convert seconds to minutes
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training:  83%|████████▎ | 5/6 [00:15<00:03,  3.09s/it]
